In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 2.3 MB/s eta 0:00:0000:0100:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 672.8 kB/s eta 0:00:0000:0100:01
  Using cached PyYAML-6.0-cp310-cp310-macosx_10_9_x86_64.whl (197 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 976.0 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 735.0 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 2.0 MB/s eta 0:00:0000:0100:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 2.3 MB/s eta 0:00:00a 0:00:01


# 1. tokenizer

- tokenizer, model: 相匹配，tokenizer outputs => model input
- Auto*Tokenizer, AutoModel*：Generic type
- tokenizer：服务于 model input
    - len(input_ids) == len(attention_mask)
    - tokenizer(test_senteces[0], ): tokenizer.__call__：encode
    - tokenizer.encode == tokenizer.tokenize + tokenizer.convert_tokens_to_ids
    - tokenizer.decode
    - tokenizer 工作的原理其实就是 tokenizer.vocab：字典，存储了 token => id 的映射关系
        - tokenizer.special_tokens_map
    - attention mask 与 padding 相匹配；

In [15]:
test_sentences = ['today is not that bad', 'today is so bad','so good']
model_name = 'distilbert-base-uncased-finetuned-sst-2-english'


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)


/Users/tangyun/opt/miniconda3/envs/rl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
batch_input = tokenizer(test_sentences, truncation=True,
                        padding=True, return_tensors='pt')
batch_input

{'input_ids': tensor([[ 101, 2651, 2003, 2025, 2008, 2919,  102],
        [ 101, 2651, 2003, 2061, 2919,  102,    0],
        [ 101, 2061, 2204,  102,    0,    0,    0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 0, 0, 0]])}

In [6]:
tokenizer.tokenize(test_sentences[0])

['today', 'is', 'not', 'that', 'bad']

In [8]:
tokenizer.encode(test_sentences[0],)

[101, 2651, 2003, 2025, 2008, 2919, 102]

In [7]:
tokenizer.convert_tokens_to_ids(tokenizer.tokenize(test_sentences[0]))

[2651, 2003, 2025, 2008, 2919]

In [9]:
tokenizer.decode([101, 2651, 2003, 2025, 2008, 2919, 102])

'[CLS] today is not that bad [SEP]'

In [11]:
tokenizer.vocab_size,tokenizer.vocab,tokenizer.special_tokens_map

(30522,
 {'wichita': 18614,
  'pascal': 17878,
  'carved': 7844,
  'opinion': 5448,
  '228': 22238,
  'troopers': 26246,
  'msc': 23794,
  '##igraphy': 23132,
  '##odies': 27391,
  '560': 21267,
  '##oge': 23884,
  'eduard': 20424,
  'sterling': 10933,
  '[unused6]': 7,
  'greatest': 4602,
  'commonwealth': 5663,
  'wrong': 3308,
  '##tead': 14565,
  '##sdale': 15145,
  '258': 24398,
  'desirable': 16166,
  'expeditions': 15014,
  'publishers': 8544,
  'gases': 15865,
  'footprint': 24319,
  'somerville': 27209,
  'chad': 9796,
  'vertex': 19449,
  'sponsoring': 29396,
  '西': 1947,
  'canary': 17154,
  'deliver': 8116,
  '[unused390]': 395,
  'escalated': 26814,
  'expensive': 6450,
  'sectors': 11105,
  'murdered': 7129,
  'cow': 11190,
  'communicate': 10639,
  'courtney': 14139,
  'screening': 11326,
  'restrict': 21573,
  'evie': 26400,
  'manitoba': 10512,
  'armenians': 20337,
  'machines': 6681,
  'somehow': 5064,
  'classified': 6219,
  'billie': 18210,
  'monograph': 24039,
  

In [12]:
tokenizer.convert_tokens_to_ids(list(tokenizer.special_tokens_map.values()))

[100, 102, 0, 101, 103]

## 2. 调用模型

In [24]:
import torch
import torch.nn.functional as F
with torch.no_grad():
    outputs = model(**batch_input)
    scores = F.softmax(outputs[0], dim=1)
    labels = torch.argmax(scores, dim=1)
    labels = [model.config.id2label[i] for i in labels.tolist()]
    print(outputs, scores, labels)

SequenceClassifierOutput(loss=None, logits=tensor([[-3.4620,  3.6118],
        [ 4.7508, -3.7899],
        [-4.1938,  4.5566]]), hidden_states=None, attentions=None) tensor([[8.4632e-04, 9.9915e-01],
        [9.9980e-01, 1.9531e-04],
        [1.5837e-04, 9.9984e-01]]) ['POSITIVE', 'NEGATIVE', 'POSITIVE']


## 3. parse output